Import necessary modules.

In [ ]:
import random
import math
import simpy

Calculate N. Last three digits of our student numbers are 210, 024 and 123.

In [ ]:
def get_N():
    s = 210 + 24 + 123
    if s > 1000:
        return s
    elif s > 10:
        return s + 1000
    else:
        return s * 300


N = get_N()

K, Lambda, Mu1 and Mu2 are already given.

In [ ]:
K = math.ceil(N / 12)

Lambda = N / 300

Mu1 = 1 / 6

Mu2 = 1 / 10

Function for calculating Mu3 as specified in the description.

Mu3 will be set with that function before running the simulation.

In [ ]:
def get_Mu3():
    r = random.uniform(1, 2)
    return 1 / (1 / Mu1 * r)


Mu3 = None  # we will set it before running the simulation

Event, FutureEventList, and SimulationEvent classes.

**Event** objects represent events. **FutureEventList** object will hold the next arrival and departure events, and the ending event if it exists. We will update it throughout the simulation and put copies of it into **SimulationEvent** objects. **SimulationEvent** objects represent each event in the simulation.

In [ ]:
class Event:
    def __init__(self, event_time, event_type, patient_index):
        self.event_time = event_time
        self.event_type = event_type
        self.patient_index = patient_index

    def __str__(self):
        return f"{self.event_type} of Patient {self.patient_index} at {self.event_time}"


class FutureEventList:
    def __init__(self, next_arrival=None, next_departure=None, simulation_end=None):
        self.next_arrival = next_arrival
        self.next_departure = next_departure
        self.simulation_end = simulation_end

    def __str__(self):
        string = "[ "
        if self.next_arrival.event_time < self.next_departure.event_time:
            string += self.next_arrival.__str__() + ", "
            string += self.next_departure.__str__()
        else:
            string += self.next_departure.__str__() + ", "
            string += self.next_arrival.__str__() + "\n"
        if self.simulation_end is not None:
            string += f", Simulation end at {self.simulation_end} ]"
        else:
            string += " ]"
        return string

    def clone(self):
        return FutureEventList(self.next_arrival, self.next_departure, self.simulation_end)


class SimulationEvent(Event):
    def __init__(self, event_time, event_type, patient_index):
        super().__init__(event_time, event_type, patient_index)
        self.fel = Fel.clone()
        self.l = hospital.count + homes.count
        self.l_hosp = hospital.count

    def __str__(self):
        return f"{self.event_type} of Patient {self.patient_index} at {self.event_time}" + "\n" + \
            f"FEL: {self.fel}" + "\n" + \
            f"Number of sick people: {self.l}" + "\n" + \
            f"Number in the hospital: {self.l_hosp}" + "\n"

Define the variables that we are going to need.

In [ ]:
Events = []              # list of events
L = []                   # number of sick people at time t
L_hosp = []              # number of people in the hospital at time t
L_home = []              # number of people in their homes at time t
Sickness_times = []      # a list of all sickness times
Fel = FutureEventList()  # fel. it will be updated throughout the simulation
env = None
hospital = None; homes = None
L_monitor_process = None; L_hosp_monitor_process = None; L_home_monitor_process = None

 ### The class definition for the patients arriving at the modeled system. 
 - Constructor: `__init__` function: When they are created, they immediatelly initiate a call (i.e. activate the call process).
 - `get_sick2` function:  This function corresponds to initial patients at the hospital, so all patients directly goes to the hospital.
 - `get_sick` function: 
    - A patient stays at home with 80% probability. His/her healing time is exponentially distributed with Mu2.
    - A patient goes to hospital with 20% probability.
       - If s/he is accepted to hospital, his/her healing time is exponentially distributed with Mu1.
       - If s/he is not accepted to hospital due to full capacity, his/her healing time is exponentially distributed with Mu3.

In [ ]:
class Patient:
    def __init__(self, index, is_initial=False):
        self.index = index
        self.name = f"Patient {index}"
        if is_initial:
            self.action = env.process(self.get_sick2())
        else:
            Events.append(SimulationEvent(env.now, "arrival", self.index))
            self.action = env.process(self.get_sick())

    def get_sick2(self):
        with hospital.request() as req:
            yield req
            sickness_time = random.expovariate(Mu1)
            Fel.next_departure = Event(env.now + sickness_time, "departure", self.index)
            yield env.timeout(sickness_time)

        Events.append(SimulationEvent(env.now, "departure", self.index))
        Sickness_times.append(sickness_time)

        inter_arrival = random.expovariate(Lambda)
        Fel.next_arrival = Event(env.now + inter_arrival, "arrival", self.index)

        Patient(self.index)

    def get_sick(self):
        rand = random.random()
        if rand < 0.8:
            with homes.request() as req:
                yield req
                sickness_time = random.expovariate(Mu2)
                Fel.next_departure = Event(env.now + sickness_time, "departure", self.index)
                yield env.timeout(sickness_time)

            Events.append(SimulationEvent(env.now, "departure", self.index))
            Sickness_times.append(sickness_time)

            inter_arrival = random.expovariate(Lambda)
            Fel.next_arrival = Event(env.now + inter_arrival, "arrival", self.index)

            Patient(self.index)
        else:
            req = hospital.request()
            results = yield req | env.timeout(0)
            if req in results:
                sickness_time = random.expovariate(Mu1)
                Fel.next_departure = Event(env.now + sickness_time, "departure", self.index)
                yield env.timeout(sickness_time)
                req.release()

                Events.append(SimulationEvent(env.now, "departure", self.index))
                Sickness_times.append(sickness_time)

                inter_arrival = random.expovariate(Lambda)
                Fel.next_arrival = Event(env.now + inter_arrival, "arrival", self.index)

                Patient(self.index)
            else:
                req.release()
                with homes.request() as req2:
                    yield req2
                    sickness_time = random.expovariate(Mu3)
                    Fel.next_departure = Event(env.now + sickness_time, "departure", self.index)
                    yield env.timeout(sickness_time)

                Events.append(SimulationEvent(env.now, "departure", self.index))
                Sickness_times.append(sickness_time)

                inter_arrival = random.expovariate(Lambda)
                Fel.next_arrival = Event(env.now + inter_arrival, "arrival", self.index)

                Patient(self.index)


`N` patients are generated with exponential distribution. initial_patient_count determines the number of patients in hospital at the beginning.

In [ ]:
def patient_generator(initial_patient_count=0):
    for i in range(1, initial_patient_count+1):
        Patient(i, True)
    for i in range(initial_patient_count+1, N+1):
        Patient(i)
        inter_arrival = random.expovariate(Lambda)
        Fel.next_arrival = Event(env.now + inter_arrival, "arrival", i)
        yield env.timeout(inter_arrival)


Defining monitors. They will keep track of number of patient in the system, in the hospital, and in the homes, respectively

In [ ]:
def L_monitor():
    while True:
        L.append(hospital.count + homes.count)
        yield env.timeout(1)

In [ ]:
def L_hosp_monitor():
    while True:
        L.append(hospital.count)
        yield env.timeout(1)

In [ ]:
def L_home_monitor():
    while True:
        L.append(homes.count)
        yield env.timeout(1)

Functions for running and resetting the simulation

In [ ]:
def reset_variables():
    global Events, L, L_hosp, L_home, Sickness_times, Fel

    Events = []
    L = []
    L_hosp = []
    L_home = []
    Sickness_times = []
    Fel = FutureEventList()


def run_simulation(initial_patient_count=0, time_limit=None, event_limit=None):
    global env, hospital, homes, L_monitor_process, L_hosp_monitor_process, L_home_monitor_process, Fel

    env = simpy.Environment()
    hospital = simpy.Resource(env, capacity=K)  # Hospital has K capacity.
    homes = simpy.Resource(env, capacity=int(1e100))  # Homes has infinite capacity.
    env.process(patient_generator(initial_patient_count))
    L_monitor_process = env.process(L_monitor())
    L_hosp_monitor_process = env.process(L_hosp_monitor())
    L_home_monitor_process = env.process(L_home_monitor())
    if time_limit is not None:
        Fel.simulation_end = time_limit
        env.run(until=time_limit)
    if event_limit is not None:
        while len(Events) < event_limit:
            env.step()

Running the simulation for 50 events

In [ ]:
random.seed(1234)
Mu3 = get_Mu3()
reset_variables()
run_simulation(event_limit=50)
# TODO printing stuff and getting model responses

Running the simulation for combinations of three different inital patient counts, three different time limits, and three different seeds

In [ ]:
for initial_patient_count in (0, math.floor(K/2), K):
    for time_limit in (1000, 10000, 100000):
        for seed in (123, 456, 789):
            random.seed(seed)
            Mu3 = get_Mu3()
            reset_variables()
            run_simulation(initial_patient_count=initial_patient_count, time_limit=time_limit)
            # TODO printing stuff and getting model responses